# Continuous Assessment 1


## Additional Libraries


In [3]:
# Add colour to text
from colorama import Fore, Style


## Utilities

Tools for input validation used throughout the notebook.


In [5]:
def positive_number_validation(num: float) -> None:
    """Ensure number is positive, throw error if not."""
    if num < 0:
        raise ValueError


def parse_input(value: str, is_int: bool) -> int | float:
    """Parse string as an int or float.

    Args:
        value (str): user input
        is_int (bool): sets if it is an int or f

    Returns:
        int|float: Converted user input.

    """
    return int(value) if is_int else float(value)


def validate_input_num(user_input: str, err_msg: str, *, is_int: bool) -> str:
    """Validate inputs to check that they are positive numbers.

    Args:
        user_input (str): user_input for the input.
        err_msg (str): error message to display on failure.
        is_int (bool): defines whether we are dealing with an integer or float.

    Returns:
        str: return the user_input if everything passes.

    """
    # Use loop to ask for input again if the input is invalid.
    while True:
        try:
            value = input(user_input)
            test_num = parse_input(value, is_int)
            positive_number_validation(test_num)
        except ValueError:
            # Give feedback with the err_msg
            print(Fore.RED + err_msg + Style.RESET_ALL + '\n')
        else:
            # Return value if input is valid
            return test_num


## Input, Output, and Processing

Week 2 Questions


### 1. Personal Information

Write a program that displays the following information:

1.  Your Name
2.  Your Address, with City, County, and ZIP CODE
3.  Your telephone number
4.  Your College degree


In [ ]:
def get_details() -> None:
    """Take user input and print details."""
    # List of details to request from user
    questions = [
        'Full Name:',
        'Street Address:',
        'City:',
        'County:',
        'Zip Code:',
        'Telephone Number:',
        'Qualifications:',
    ]

    # String comprehension to take input for question list, create new list
    print('Please input the following info: \n')
    personal_details = [input(question) for question in questions]

    # Print user data with info titles using zip and loop
    for question, item in zip(questions, personal_details, strict=True):
        print(f'{question:<18} {item}')

In [11]:
get_details()

Please input the following info: 

Full Name:         Jacob SMith
Street Address:    123 sesame st.
City:              dublin
County:            dublin
Zip Code:          112345
Telephone Number:  2445667
Qualifications:    Degree


### 2. Total Purchase

A customer in a store is purchasing five items. Write a program that asks for the price of each item, then display the subtotal of the sales, the amount of sales tax, and the total.

Assume sales tax is 7%.


In [12]:
def calculate_total(num_items: int = 5) -> None:
    """Take input of item prices and calculate total sales cost.

    Args:
        num_items (int, optional): number of purchase items. Defaults to 5.

    """
    print(
        'Please input item prices with two decimal points, ex. 12.95.\n'
        'Long decimals will be rounded to two decimal points.\n',
    )

    # Initiate list to collect prices
    prices = []
    err_msg = 'Please input positive numbers with two decimals, ex.12.95.'

    # Ask for item prices for as many items are requested.
    for num in range(num_items):
        user_input = f'Price of item {num + 1}:'
        # Loop that checks the item info is correctly input.
        value = validate_input_num(user_input, err_msg, is_int=False)
        item = round(value, 2)
        prices.append(item)

    # Calculates the subtotal by adding prices together.
    subtotal = 0
    for price in prices:
        subtotal += price

    # Calculate tax and then total.
    sales_tax = round(subtotal * 7 / 100, 2)
    grand_total = subtotal + sales_tax

    # Displays prices to user with loop.
    for index, price in enumerate(prices):
        # Format string, show numbers with two decimal points
        print(f'{"Item"}{index + 1}{":":<8}{price:.2f}')
    print(f'{"Subtotal:":<13}{subtotal:.2f}')
    print(Fore.YELLOW + f'{"7% Sales Tax:"} {sales_tax:.2f}' + Style.RESET_ALL)
    print(Fore.GREEN + f'{"Grand Total:"} {grand_total:.2f}' + Style.RESET_ALL)

In [16]:
calculate_total()

Please input item prices with two decimal points, ex. 12.95.
Long decimals will be rounded to two decimal points.

Please input positive numbers with two decimals, ex.12.95.

Please input positive numbers with two decimals, ex.12.95.

Item1:       10.00
Item2:       12.00
Item3:       23.57
Item4:       89.00
Item5:       55555.56
Subtotal:    55690.13
7% Sales Tax: 3898.31
Grand Total: 59588.44


### 3. Distance Travelled

The distance a car drives down the interstate can be calculated with the formula:
Distance = Speed x Time

A car travels at 70mph. Write a program that displays the following:

1. The distance the car will travel in 6 hours
2. The distance the car will travel in 10 hours
3. The distance the car will travel in 15hours

I have provided two version as an answer:
The first is a straightforward calculation based on the question requests.
The seconds allows for varied input and produces the resullt.


In [ ]:
def calc_distance(speed: int = 70) -> None:
    """Calculate the distance of a car traveling at a set speed.

    Args:
        speed (int, optional): Speed at which the car travels. Defaults to 70.

    """
    # Takes a list of times, then calculates distance and shows it to the user.
    times = [6, 10, 15]
    for hrs in times:
        distance = hrs * speed
        print(f'The car will travel {distance} miles in {hrs} hours.')


def calculate_distance_with_input() -> None:
    """Calculate distance based on time and speed."""
    # The following version requests input for the desired time and speed
    print(
        'This app will calculate the distance based on the time and speed.\n',
    )
    user_input_one = 'Time in hours:'
    user_input_two = 'Speed in mph:'
    err_msg = 'Please input positive whole numbers, ex. 12.'
    value_1 = validate_input_num(user_input_one, err_msg, is_int=True)
    value_2 = validate_input_num(user_input_two, err_msg, is_int=True)
    time = value_1
    speed = value_2

    distance = time * speed
    print(f'The car will travel {distance} miles in {time} hours.')

In [17]:
# Version 1 - simple calculation
calc_distance()

The car will travel 420 miles in 6 hours.
The car will travel 700 miles in 10 hours.
The car will travel 1050 miles in 15 hours.


In [20]:
# Version 2 - with input
calculate_distance_with_input()

This app will calculate the distance based on the time and speed.

Please input positive whole numbers, ex. 12.

Please input positive whole numbers, ex. 12.

Please input positive whole numbers, ex. 12.

Please input positive whole numbers, ex. 12.

The car will travel 36 miles in 12 hours.


### 4. Miles per Gallons

A cars MPG can be calculated with the following formula

MPG = Miles Driven / Gallons of Diesel used

Write a program that asks the user for the number of miles driven and gallons of diesel used.
It should calculate the car's MPG and display the result.


In [ ]:
def calculate_mpg() -> None:
    """Request user input to calculate Miles Per Gallon."""
    # Input validation prompts and error message.
    print('This app calculates the miles per gallon of a vehicle.\n')
    user_input_one = 'How many miles were driven?'
    user_input_two = 'How many gallons of diesel were used?'
    err_msg = 'Please input positive whole numbers, ex. 12.'

    # Calculates mpg and displays it to the user.
    miles = validate_input_num(user_input_one, err_msg, is_int=True)
    gallons = validate_input_num(user_input_two, err_msg, is_int=True)
    print(f'The miles per gallon is {miles / gallons:.2f}.')

In [24]:
calculate_mpg()

This app calculates the miles per gallon of a vehicle.
Please input positive whole numbers, ex. 12.

The miles per gallon is 0.86.


### 5. Male and Female percentages

Write a program that asks the user for the number of Males and the number of Females registered for the class. The program should display the percentage of Males and Females in the class.

Example:

8 Males and 12 Females in class.

There are 20 students in the class.

The percentage of Males can be calculated as 8 / 20 = .4 or 40%

The percentage of Females can be calculated as 12 / 20 = .6 or 60%


In [ ]:
def calculate_student_percentages() -> None:
    """Calculate the percentage of female and male students in a class."""
    print(
        'This app calculates the percentage of female and male students.\n'
        'Percentages will be rounded to two decimal points.\n',
    )
    # Input validation prompts and error message.
    user_input_one = 'How many females are in the class?'
    user_input_two = 'How many males are in the class?'
    err_msg = 'Please input positive whole numbers, ex. 12.'

    # Calculates mpg and displays it to the user.
    females = validate_input_num(user_input_one, err_msg, is_int=True)
    males = validate_input_num(user_input_two, err_msg, is_int=True)
    total = females + males
    print(
        f'The total number of student is {total}.\n'
        f'The percentage of females is {females / total:.2f} or '
        f'{females / total:.2%}.\n'
        f'The percentage of males is {males / total:.2f}. or '
        f'{males / total:.2%}.',
    )


In [30]:
calculate_student_percentages()

This app calculates the percentage of female and male students.
Percentages will be rounded to two decimal points.

The total number of student is 118.
The percentage of females is 0.33 or 33.05%.
The percentage of males is 0.67. or 66.95%.


### 6. Stock Transaction Program

Tom purchased some stock in DBS Software.
Here are the details.

1. The number of shares that Tom purchased was 2,000
2. When Tom purchased the stock , he paid €40.0 per share.
3. Tom paid his stockbroker a commission that amounted to 3% of the amount he paid for the stock.

Two weeks later , Tom sold the stock. Here are the details

1. The number of shares that Tom sold was 2,000
2. When Tom sold the stock , he sold for €42.75 per share.
3. Tom paid his stockbroker another commission that amounted to 3% of the amount he received for the stock.

Write a program that displays the following information.

1. The amount of money Tom paid for the stock
2. The amount of commission Tom paid his broker when he bought the stock.
3. The amount for which Tom sold the stock
4. The amount of commission Tom paid his broker when he sold the stock.
5. Display the amount of money that Tom had left when he sold the stock and paid his broker(both times). If this amount is positive, then Tom made a profit. If the amount is negative, then Tom lost money.


In [ ]:
def stock_calculator(
    buy_shares: int = 2000,
    buy_price: float = 40.0,
    commission_percent: int = 3,
    sell_price: float = 42.75,
    sell_shares: int = 2000,
) -> None:
    """Calculate profit, loss and costs from stock.

    Args:
        buy_shares (int, optional): Number of shares bought. Defaults to 2000.
        buy_price (float, optional): Cost of shares bought. Defaults to 40.0.
        commission_percent (int, optional): Percentage commission on buy/sell.
                                            Defaults to 3.
        sell_price (float, optional): Selling price of shares.
                                      Defaults to 42.75.
        sell_shares (int, optional): Number of shares sold. Defaults to 2000.

    """
    print(
        'This program calculates profit, loss, and costs from stock '
        'purchases and sales.\n',
    )
    # Use parameters to calculate different values
    purchase_cost = buy_shares * buy_price
    purchase_commission = purchase_cost * commission_percent / 100
    sale_total = sell_shares * sell_price
    sale_commission = sale_total * commission_percent / 100
    total_money = sale_total - purchase_commission - sale_commission
    net_profit = total_money - purchase_cost

    # Display values to user.
    print(f'The total cost of the purchase is €{purchase_cost:.2f}')
    print(
        'The total purchase commission is '
        + Fore.RED
        + f'€{purchase_commission:.2f}'
        + Style.RESET_ALL,
    )
    print(
        'The total income from the sale is '
        + Fore.GREEN
        + f'€{sale_total:.2f}'
        + Style.RESET_ALL,
    )
    print(
        'The total sales commission is '
        + Fore.RED
        + f'€{sale_commission:.2f}'
        + Style.RESET_ALL,
    )

    print(f'The total sale less commission is €{total_money:.2f}')
    print(
        'The total net profit is '
        + Fore.GREEN
        + f'€{net_profit:.2f}'
        + Style.RESET_ALL,
    )

In [ ]:
# Default arguments can be changed to calculate other amounts.
stock_calculator(buy_shares=2000)

This program calculates profit, loss, and costs from stock purchases and sales.

The total cost of the purchase is €80000.00
The total purchase commission is €2400.00
The total income from the sale is €85500.00
The total sales commission is €2565.00
The total sale less commission is €80535.00
The total net profit is €535.00


## Data Representation and Program Control

For the questions and answers, please refer to:  
[3. Distance Travelled](#3-distance-travelled)  
[4. Miles per Gallons](#4-miles-per-gallons)  
[5. Male and Female percentages](#5-male-and-female-percentages)  
[6. Stock Transaction Program](#6-stock-transaction-program)


## Program Control and Decision


### 1. Bug Collector

A bug collector collects bugs every day for five days. Write a program that keeps a running total of the number of bugs collected during the five days. The loop should ask for the number of bugs collected for each day, and when the loop is finished, the program should display the total number of bugs collected.


In [ ]:
def calculate_bugs(num_days: int = 5) -> None:
    """Take input of item prices and calculate total bugs collected.

    Args:
        num_days (int, optional): number of days collect bugs. Defaults to 5.

    """
    print(
        'Please input the number of bugs collected for each day.\n'
        'Please input whole numbers, ex. 12.\n',
    )

    # Initiate total to use for bugs
    total = 0
    err_msg = 'Please input positive whole numbers, ex.12.'

    # Ask for bug quantity for as many days are requested.
    for num in range(num_days):
        user_input = f'Bugs collected on Day {num + 1}:'
        # Loop that checks the item info is correctly input.
        value = validate_input_num(user_input, err_msg, is_int=True)
        # Add number to total
        total += value

    print(f'Total number of bugs collected: {total}')

In [22]:
calculate_bugs()

Please input the number of bugs collected for each day.
Please input whole numbers, ex. 12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Please input positive whole numbers, ex.12.

Total number of bugs collected: 24


### 2. Calories Burned

Running on a particular treadmill you burn 4.2 calories per minute. Write a program that uses a loop to display the number of calories burned after 10, 15, 20, 25, and 30 minutes.


In [ ]:
def calculate_calories(rate: float = 4.2) -> None:
    """Calculate how many calories will be burned over time.

    Args:
        rate (float, optional): Calorie burn rate. Defaults to 4.2.

    """
    # Times to calculate
    time = [10, 15, 20, 25, 30]
    for minutes in time:
        print(
            f'You will burn {rate * minutes:.2f} calories in'
            ' {minutes} minutes.',
        )

In [28]:
calculate_calories()

You will burn 42.00 calories in 10 minutes.
You will burn 63.00 calories in 15 minutes.
You will burn 84.00 calories in 20 minutes.
You will burn 105.00 calories in 25 minutes.
You will burn 126.00 calories in 30 minutes.


### 3. Lap Times

Write a program that asks the user to enter the number of times that they have run around a racetrack, and then uses a loop to prompt them to enter the lap time for each of their laps. When the loop finishes, the program should display the time of their fastest lap, the time of their slowest lap, and their average lap time.


In [ ]:
def calculate_lap_times() -> None:
    """Calculate fastest, slowest and average lap times of a runner."""
    print('This app calculates stats based on your lap times.\n')
    # Take and validate input
    user_input = (
        'How many laps have you run?\n'
        'Please input positive whole numbers, ex. 12.'
    )

    err_msg = 'Please input positive whole numbers, ex. 12.'

    laps = validate_input_num(user_input, err_msg, is_int=True)
    # Initalise list of times
    lap_times = []

    # Loop as many times as laps ran and validate input.
    for lap in range(laps):
        lap_input = (
            f'What is the lap time in minutes for lap {lap + 1}?\n'
            'Please input positive numbers with two decimals, ex. 12.95.'
        )
        lap_err = 'Please input positive numbers with two decimals, ex. 12.95.'
        time = validate_input_num(lap_input, lap_err, is_int=False)
        lap_times.append(time)
    # Sort times from lowest to highest to print answers easily.
    lap_times.sort()
    print(
        Fore.GREEN
        + f'Your fastest lap time is {lap_times[0]:.2f}m.'
        + Style.RESET_ALL,
    )
    # Calculate total of times and average it.
    print(
        Fore.YELLOW
        + f'Your average lap time is {sum(lap_times) / laps:.2f}m.'
        + Style.RESET_ALL,
    )
    print(
        Fore.RED
        + f'Your slowest lap time is {lap_times[-1]:.2f}m.'
        + Style.RESET_ALL,
    )


In [ ]:
calculate_lap_times()

This app calculates stats based on your lap times.
Your fastest lap time is 1.00m.
Your average lap time is 5.00m.
Your slowest lap time is 9.00m.


### 4. Distance Traveled

follows: The distance a vehicle travels can be calculated:  
speed × time = distance  
For example, if a train travels 40 miles per hour for three hours, the distance traveled is 120 miles. Write a program that asks the user for the speed of a vehicle (in miles per hour) and the number of hours it has traveled. It should then use a loop to display the distance the vehicle has traveled for each hour of that time period.


In [ ]:
def calculate_distance_per_hour() -> None:
    """Display a table of distance covered per hour."""
    print(
        'This app calculates the distance per hour '
        'based on the time and speed.\n',
    )
    user_input_one = 'Time in hours:'
    user_input_two = 'Speed in mph:'
    err_msg = 'Please input positive whole numbers, ex. 12.'
    value_1 = validate_input_num(user_input_one, err_msg, is_int=True)
    value_2 = validate_input_num(user_input_two, err_msg, is_int=True)
    time = value_1
    speed = value_2

    print(f'{"Hour":<10} Distance')
    print('-------------------')
    for hour in range(time):
        print(f'{hour + 1:<10} {speed * (hour + 1)}')

In [47]:
calculate_distance_per_hour()

This app calculates the distance per hour based on the time and speed.

Hour       Distance
-------------------
1          8
2          16
3          24
4          32
5          40
6          48
7          56
8          64


### 5. Average Rainfall

Write a program that uses nested loops to collect data and calculate the average rainfall over a period of years. The program should first ask for the number of years. The outer loop will iterate once for each year. The inner loop will iterate twelve times, once for each month. Each iteration of the inner loop will ask the user for the inches of rainfall for that month. After all iterations, the program should display the number of months, the total inches of rainfall, and the average rainfall per month for the entire period.


In [ ]:
def calculate_rainfall() -> None:
    """Calculate statistics based on amount of rainfall."""
    # Dictionary of lists, so I can use the month name and store multiple
    # rainfall values in each to calculate the average for each month later.
    months = {
        'January': [],
        'February': [],
        'March': [],
        'April': [],
        'May': [],
        'June': [],
        'July': [],
        'August': [],
        'September': [],
        'October': [],
        'November': [],
        'December': [],
    }

    print(
        'This app displays statistic for rainfall in a given period.\n'
        'Please input positive whole numbers, ex. 12.',
    )

    # take and validate user input.
    user_input = 'What is the period in years to calculate rainfall for?'
    err_msg = 'Please input positive whole numbers, ex. 12.'

    years = validate_input_num(user_input, err_msg, is_int=True)
    total = 0
    print(
        'For inches of rainfall, please input '
        'positive numbers with two decimals, ex. 12.95.',
    )

    # Nested list to ask for month by year.
    for year in range(years):
        for key, value in months.items():
            month_input = f'How much rain fell in {key} of Year {year + 1}?'
            rain_err = (
                'Please input positive numbers with two decimals, ex. 12.95.'
            )
            rainfall = validate_input_num(month_input, rain_err, is_int=False)
            # Add to total rainfall.
            total += rainfall

            # Add rainfall to specific month list.
            value.append(rainfall)

    print(f'Total months in period: {12 * years}')
    print(f'Total rainfall in period: {total:.2f}\n')
    print('Average rainfall per month:')
    print(f'{"Month":<15} Average')
    print('-----------------------')

    # Calculate and display average rainfall per month.
    for key, value in months.items():
        print(f'{key:<15} {sum(value) / years:.2f}')

In [13]:
calculate_rainfall()

This app displays statistic for rainfall in a given period.
Please input positive whole numbers, ex. 12.
For inches of rainfall, please input positive numbers with two decimals, ex. 12.95.
Total months in period: 24
Total rainfall in period: 1086.00

Average rainfall per month:
Month           Average
--------------------------
January         15.0
February        25.0
March           35.0
April           45.0
May             55.0
June            65.0
July            75.0
August          85.0
September       95.0
October         15.0
November        16.0
December        17.0


## Homework


In [ ]:
def check_baggage(max_weight: int = 15) -> None:
    """Check the baggage weight against weight limits and gives a response.

    Args:
        max_weight (int, optional): the limit for carry-ons. Defaults to 15.

    """
    print('This checks if you are able to carry-on or check your baggage.')
    user_input = 'How heavy is your bag? Please enter the format "12.00"'
    err_msg = 'Please input positive numbers with two decimals, ex.12.95.'
    value = validate_input_num(user_input, err_msg, is_int=False)

    if value > max_weight:
        print(
            Fore.RED
            + 'Sorry, your baggage exceeds our maximum carry-on limit '
            f'of {max_weight}kg. It will have to be checked.'
            + Style.RESET_ALL,
        )
    else:
        print(Fore.GREEN + 'Thank you and welcome aboard!' + Style.RESET_ALL)


In [ ]:
check_baggage()

Thank you and welcome aboard!


In [ ]:
def calculate_tax() -> None:
    """Calculate net income after finding tax bracket."""
    user_input = 'What is your income? Please enter the format "12000.00"'
    err_msg = 'Please input positive numbers with two decimals, ex. 12.95.'
    value = validate_input_num(user_input, err_msg, is_int=False)

    tax_bracket_one = 20000
    tax_bracket_two = 40000
    tax_bracket_three = 80000

    if value < tax_bracket_one:
        tax = 0
    elif value < tax_bracket_two:
        tax = 0.2
    elif value < tax_bracket_three:
        tax = 0.4
    else:
        tax = 0.55

    tax_total = value * tax
    net_income = value - tax_total

    print(f'Your net income is €{net_income:.2f}.')

In [ ]:
calculate_tax()

Your net income is €38250.00.


In [ ]:
def calculate_penny_salary(pay_rate=0.01):
    user_input = 'How many days are we calculating pay for?'
    err_msg = 'Please input positive whole numbers, ex. 12.'

    # Calculates mpg and displays it to the user.
    days = validate_input_num(user_input, err_msg, is_int=True)
    pay = 0
    print(f'{"Day":<6}| Total Pay')
    print('---------------------------------')
    for day in range(days):
        pay += pay_rate
        pay_rate = pay_rate * 2
        print(f'{day + 1:<6}| ${pay:.2f}')
    print('---------------------------------')


In [ ]:
calculate_penny_salary()

Day   | Total Pay
---------------------------------
1     | $0.01
2     | $0.03
3     | $0.07
4     | $0.15
5     | $0.31
6     | $0.63
7     | $1.27
8     | $2.55
9     | $5.11
10    | $10.23
11    | $20.47
12    | $40.95
---------------------------------


In [ ]:
def application_questions() -> None:
    """Check if applicant meets minimum requirements for loan."""
    user_input_one = 'What is your salary?'
    user_input_two = 'How many years of experience do you have?'
    err_msg = 'Please input positive numbers, ex. 12.5'

    salary_limit = 30000
    year_limit = 2
    salary = validate_input_num(user_input_one, err_msg, is_int=False)
    years_on_job = validate_input_num(user_input_two, err_msg, is_int=False)

    if salary >= salary_limit:
        if years_on_job >= year_limit:
            print(Fore.GREEN + 'You qualify for the loan.' + Style.RESET_ALL)
        else:
            print(
                Fore.RED
                + 'You must have been on your current job for at least two'
                ' years to qualify' + Style.RESET_ALL,
            )
    else:
        print(
            Fore.RED
            + 'You must earn at least 30,000 per year to qualify.'
            + Style.RESET_ALL,
        )

In [16]:
application_questions()

You must have been on your current job for at least two years to qualify


In [ ]:
def test_score():
    user_input = 'What is the test score?'
    err_msg = 'Please input positive whole numbers, ex. 12.'

    while True:
        score = validate_input_num(user_input, err_msg, is_int=True)
        if score > 100:
            print('Please input a number less than 100.')
        else:
            break

    if score >= 90:
        grade = 'A'
    elif score >= 80:
        grade = 'B'
    elif score >= 70:
        grade = 'C'
    elif score >= 60:
        grade = 'D'
    else:
        grade = 'F'

    print(f'Your grade is {grade}')

In [20]:
test_score()

Please input a number less than 100.
Your grade is F
